In [2]:
# In[8]:
    import math
import numpy as np
import cupy as cp
pool = cp.cuda.MemoryPool(cp.cuda.malloc_managed)
cp.cuda.set_allocator(pool.malloc)
import sys
import os
from scipy.stats import norm
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.path.dirname('__file__'), '..'))

from modulation.modulation import QAMModem

In [3]:
def add_AWGN_GPU(constellation,No):
  # AWGN雑音の生成
  noise = cp.random.normal(0, math.sqrt(No / 2), (len(constellation)))           + 1j * cp.random.normal(0, math.sqrt(No / 2), (len(constellation)))

  # AWGN通信路 = 送信シンボル間干渉が生じないような通信路で送信
  RX_constellation = constellation + noise

  # 以下のprint関数の出力を表示すると、Noとほぼ一致するはず
  #print(cp.dot(noise[0, :], cp.conj(noise[0, :]))/bit_num)

  return RX_constellation

In [29]:
def make_AMI(EsNodB,M):
  
    EsNo = 10 ** (EsNodB / 10)
    No=1/EsNo
    count_num=100000

    #make info matrices
    info=cp.random.randint(0,M,count_num)

    #make constellation
    modem=QAMModem(M)
    tmp=modem.code_book
    symbol=cp.zeros(M,dtype=complex)
    for i in tmp:
        symbol[modem.bin2de(i)]=tmp[i]
    
    ps=np.sum(abs(symbol)**2)/M
    #print(ps)
    if ps>1:
        
        print(M,"symbol error!")
    
    mat_symbol=cp.tile(symbol,(count_num,1))
    const=cp.take_along_axis(mat_symbol,info[:,None],axis=1)[:,0]

    #if cp.any(symbol==const)!=True:
        #print("error")
        #print(symbol)
        #print(const)

    RX_const=add_AWGN_GPU(const,No)
        
    num=cp.sum(cp.exp(-1*cp.abs(np.tile(RX_const,(len(symbol),1))-symbol.reshape(-1,1))**2/No),axis=0)
    
    den=cp.exp(-1*cp.abs(RX_const-const)**2/No)
    H=cp.sum(cp.log2(num/den))
    H/=count_num
    
    res=math.log2(M)-H
    return res

In [31]:
#M=[4,16,64,256,1024,4096]
#for m in M:
    #print(make_AMI(0,m))

0.9752883772319529
0.9892223959564523
64 symbol error!
0.9907625371097479
0.9964522810363885
0.9986871430231954
0.9856051618928152


In [103]:
def make_BMI(EsNodB,M):
    EsNo = 10 ** (EsNodB / 10)
    No=1/EsNo
    count_num=1#00000000

    #make info matrices
    info=cp.random.randint(0,M,count_num)
    #rint(info)
    #info=cp.zeros(count_num,dtype=int)
    #make constellation
    modem=QAMModem(M)
    tmp=modem.code_book
    symbol=cp.zeros(M,dtype=complex)
    for i in tmp:
        symbol[modem.bin2de(i)]=tmp[i]
        #print(modem.bin2de(i))
        #print(tmp[i])
        #print("next")

    mat_symbol=cp.tile(symbol,(count_num,1))
    const=cp.take_along_axis(mat_symbol,info[:,None],axis=1)[:,0]
    #print(const)
    RX_const=add_AWGN_GPU(const,No)

    #bitごとの0のシンボルと1のシンボルを出す
    ones=cp.array(modem.ones)
    zeros=cp.array(modem.zeros)
    ones_zeros=cp.stack([zeros,ones])
    #print(ones_zeros)
    #print(ones)
    #print(zeros)
    #for a,b in zip(ones,zeros):
        #print("zip")
        #print(a,b)
    #print(info)
    #print(ones)
    H=0
    res=0
    for i in range(0,int(math.log2(M)))[::-1]:
        ith_bits=((info)//(2**i))%2
        #print("bits",ith_bits[0:30])
        #print(const[0:10])
        
        ones_zeros_i=ones_zeros[:,int(math.log2(M))-i-1,:] #ここのインデックスが間違っていた
        #print(ones_zeros.shape)#(2,log2(M),M/2)
        #print(ones_zeros_i.shape)#(2,M/2)
        
        #print(ones_zeros_i) #check#ここが間違っていた
        
        mat_ones_zeros_i=np.tile(ones_zeros_i,(len(ith_bits),1,1))
        #print(mat_ones_zeros_i.shape) #(count_num,2,M/2)
        #print(mat_ones_zeros_i[0])
        
        res_ones_zeros_i=cp.take_along_axis(mat_ones_zeros_i,ith_bits[:,None,None],axis=1)
        #print(res_ones_zeros_i.shape) #(count_num,1,M/2)
        #print(res_ones_zeros_i)
        
        res_ones_zeros_i=res_ones_zeros_i[:,0,:]
        #print(res_ones_zeros_i.shape) #(count_num,M/2)
        
        res_ones_zeros_i=res_ones_zeros_i.T
        #print(RX_const[0])
        num=cp.sum(cp.exp(-1*cp.abs(np.tile(RX_const,(len(symbol),1))-symbol.reshape(-1,1))**2/No),axis=0)
        den=cp.sum(cp.exp(-1*cp.abs(np.tile(RX_const,(len(ones[0]),1))-res_ones_zeros_i)**2/No),axis=0)
        H=cp.sum(cp.log2(num/den))
        tmp=1-H/count_num
        #print(tmp)
        res+=tmp

    #H/=count_num
    #res=math.log2(M)-H
    #print(res)
    return res

In [104]:
SNR_range=np.arange(0,10,0.5)
M_list=[4,16,256]
#print(SNR_range)

BMI_list=np.zeros(len(SNR_range))

for M in M_list:
    AMI_list=np.zeros(len(SNR_range))
    filename="AMI_{}QAM".format(M)
    for i,EsNodB in enumerate(SNR_range):
        AMI_list[i]=make_AMI(EsNodB,M)
        
        with open(filename,'w') as f:
            for i in range(len(SNR_range)):
                print(str(SNR_range[i]),str(AMI_list[i]),file=f)
                
    BMI_list=np.zeros(len(SNR_range))
    filename="BMI_{}QAM".format(M)
    for i,EsNodB in enumerate(SNR_range):
        BMI_list[i]=make_BMI(EsNodB,M)
        
        with open(filename,'w') as f:
            for i in range(len(SNR_range)):
                print(str(SNR_range[i]),str(BMI_list[i]),file=f)


    

In [20]:
def C_inv(I):
    def A(c):
        return (-5+24*np.log(2)*c+2*(13+12*np.log(2)*c*(12*np.log(2)*c-5))**(1/2))**(1/3)

    def W0(x):
        '''
        Lambert W function
        reference:
        On the lambert W function
        (3.1)
        '''
        def a(n):
            ((-n)**(n-1)/np.math.factorial(n))*x**n
            
            
        res=0
        for i in range(1000):
            res+=a(i)
        return res
    
    if I>1 or I<0:
        print("I is err")
            
    #print("use new funcß")   
    C1=0.055523
    C2=0.721452
    C3=0.999983
    H21=1.396634
    H22=0.872764
    H23=1.148562
    H31=1.266967
    H32=0.938175
    H33=0.986830
    alpha=1.16125
    
    if I<C1:
        gamma=1/4*(1-3/A(I)+A(I))
        
    elif I<C2:
        gamma=(-1/H21*np.log(1-I**(1/H23)))**(1/H22)
        
    elif I<C3:
        gamma=(-1/H31*np.log(1-I**(1/H33)))**(1/H32)
    
    else:
        gamma=1/2*W0(2*(alpha/(1-I))**2)
    
    return gamma

In [21]:
def J_inv(I):
    if I>1 or I<0:
            print("I is err")
        
    a1=1.09542
    b1=0.214217
    c1=2.33727
    a2=0.706692
    b2=0.386013
    c2=-1.75017
    I_thresh=0.3646
    
    if I<I_thresh:
        sigma=a1*I**2+b1*I+c1*I**(1/2)
    else:
        sigma=-a2*np.log(b2*(1-I))-c2*I
        
    gamma=sigma**2/8
    #gamma_dB=10*math.log10(gamma)
    return gamma

In [44]:
import numpy as np

C_list=np.arange(0,1,0.03)
C_list_upper=np.arange(0.983,0.999,0.003)
snr=np.zeros(len(C_list))
snr_remaining=np.zeros(len(C_list_remaining))
#print(C_list)
for i in range(len(C_list)):
    snr[i]=J_inv(C_list[i])

for i in range(len(C_list_remaining)):
    #print(C_list_remaining[i])
    snr_remaining[i]=J_inv(C_list_remaining[i])
    print(snr_remaining[i])
    

#change snr to db

snr=10*np.log10(snr)
print(snr_remaining)
snr_remaining=10*np.log10(snr_remaining)
print("a",snr_remaining)
C_list=np.concatenate([C_list,C_list_remaining])
snr=np.concatenate([snr,snr_remaining])
print(len(snr))

filename="J_inverse_function"
with open(filename,'w') as f:

        print("#EsNodB,BLER,BER",file=f)  
        for i in range(len(C_list)):
            print(str(snr[i]),str(C_list[i]),file=f)
    
    
    

4.004158798693084
4.376515027173717
4.989372857585063
6.666295610643219
[4.0041588  4.37651503 4.98937286 6.66629561]
a [6.02511293 6.41128424 6.9804596  8.23884568]
38


/tmp/ipykernel_982859/1547432854.py:19: RuntimeWarning: divide by zero encountered in log10
  snr=10*np.log10(snr)
